<a href="https://colab.research.google.com/github/Joshua250304/JPMC-Research-Project/blob/main/Commodity_Storage_Contract_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

print("Please upload the 'UnsampledData.csv' file.")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

# To verify the file is uploaded, you can list the files in the current directory
# !ls

Please upload the 'UnsampledData.csv' file.


Saving UpsampledData.csv to UpsampledData.csv
User uploaded file "UpsampledData.csv" with length 68365 bytes


In [ ]:
import numpy as np
import pandas as pd
from datetime import date

data = pd.read_csv('UpsampledData.csv')

def get_gas_price(query_date):
    """
    Retrieves the gas price for a given date from the 'data' DataFrame.
    """
    # Ensure 'Dates' column is datetime objects for proper comparison
    data['Dates'] = pd.to_datetime(data['Dates'])
    query_datetime = pd.to_datetime(query_date)

    # Find the price for the exact date. If not found, use the closest preceding date.
    # For simplicity, we'll look for an exact match first.
    price_row = data[data['Dates'] == query_datetime]

    if not price_row.empty:
        return price_row['Prices'].iloc[0]
    else:
        # If exact date not found, find the closest date before the query date
        # This is a simplification; a more robust model might interpolate or use a forecasting model
        closest_date_row = data[data['Dates'] <= query_datetime].sort_values(by='Dates', ascending=False)
        if not closest_date_row.empty:
            return closest_date_row['Prices'].iloc[0]
        else:
            raise ValueError(f"No gas price available for or before {query_date}")

def price_storage_contract(injection_date, withdrawal_date, volume, storage_rate, injection_fee, withdrawal_fee):
    """
    Calculates the net value of a gas storage contract.

    Parameters:
    - injection_date (str): 'YYYY-MM-DD' when gas is bought/stored.
    - withdrawal_date (str): 'YYYY-MM-DD' when gas is sold/removed.
    - volume (int): Million Metric British Thermal Units (MMBtu).
    - storage_rate (float): Monthly cost to store the gas.
    - injection_fee (float): Cost per unit to put gas in.
    - withdrawal_fee (float): Cost per unit to take gas out.
    """

    # 1. Get prices from Task 1 model
    buy_price = get_gas_price(injection_date)
    sell_price = get_gas_price(withdrawal_date)

    # 2. Calculate time elapsed in months
    d1 = pd.to_datetime(injection_date)
    d2 = pd.to_datetime(withdrawal_date)
    months_stored = (d2.year - d1.year) * 12 + (d2.month - d1.month)

    if months_stored < 0:
        return "Error: Withdrawal date must be after injection date."

    # 3. Calculate financial components
    total_purchase_cost = volume * buy_price
    total_sale_revenue = volume * sell_price

    total_storage_cost = months_stored * storage_rate
    total_injection_cost = volume * injection_fee
    total_withdrawal_cost = volume * withdrawal_fee

    # 4. Final Valuation
    net_value = total_sale_revenue - total_purchase_cost - total_storage_cost - total_injection_cost - total_withdrawal_cost

    return round(net_value, 2)

# --- Example Scenario ---
# Injecting 100,000 MMBtu in Summer (July), withdrawing in Winter (January)
contract_value = price_storage_contract(
    injection_date='2025-07-01',
    withdrawal_date='2026-01-01',
    volume=100000,
    storage_rate=5000,      # $5,000 per month
    injection_fee=0.10,     # $0.10 per MMBtu
    withdrawal_fee=0.10     # $0.10 per MMBtu
)

print(f"The estimated value of the storage contract is: ${contract_value}")

The estimated value of the storage contract is: $-10994.09
